# Домашнее задание №2.2

**Задача** – написать функцию, которая принимает на вход `dataframe` (после `join`), а на выходе дает средний `pFound` по всем query
    



*   Запрещается использовать циклы `for` для расчет метрики (как полностью, так и ее частей)
*   Усложнение, если задача показалась легкой - попробуйте обойтись без `groupby`



## Подготовка данных

In [1]:
import os
import pandas as pd
import numpy as np

from pathlib import Path
from tqdm.auto import tqdm

In [2]:
PATH = Path('/Users/nikitaborisov/Desktop/yandex_cup')

In [3]:
# считываем данные
qid_query = pd.read_csv(f"{PATH}/open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv(f"{PATH}/open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv(f"{PATH}/open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

## Решение Yandex Cup

In [5]:
def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)
     

def pfound(group):
    # Максимальный рейтинг хоста
    max_by_host = group.groupby("hostid")["rating"].max() 
    
    # Топ-10 хостов с наивысшим рейтингом
    top10 = max_by_host.sort_values(ascending=False)[:10] 
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

In [10]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # Группировка по qid и вычисление pfound
qid_max = qid_pfound.idxmax() # qid с максимальным pfound

In [11]:
qid_query[qid_query["qid"] == qid_max]

,qid,query
12,295761,гугл переводчик


In [12]:
# считаем среднее pFound
qid_pfound.mean() 

0.4603173929969002

## Решение без цикла

In [52]:
def pFound(
    df: pd.DataFrame,
    k: int = 10,
    p_break: float = 0.15) -> float:
    
    # найдем максимально релевантный документ в каждом хосте и отсортируем хосты в каждом запросе по убыванию
    qid_hostid_max_rating = df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    sorted_qid = qid_hostid_max_rating.sort_values(['qid', 'rating'], ascending=False)
    
    # найдем топ-k хостов в каждом запросе
    sorted_qid_top_k = sorted_qid.groupby(['qid']).head(k)
    
    top_k = sorted_qid_top_k.copy()
    
    # добавим вспомогательные столбцы с промежуточными результатами расчета
    top_k['1 - pBreak'] = 1 - p_break
    top_k['1 - pRel'] = (1 - top_k['rating']).shift(1)
    top_k['sorted_num'] = top_k.groupby('qid').cumcount()
    
    # изменим первые значения на (1) 
    top_k.loc[top_k['sorted_num'] == 0, ['1 - pRel', '1 - pBreak']] = 1
     
    # рассчитаем pLook
    top_k['prod'] = top_k['1 - pRel'] * top_k['1 - pBreak']
    top_k['pLook'] = top_k.groupby('qid')['prod'].cumprod()
    
    # найдем значение pFound для каждого host, а затем для qid
    top_k['pFound'] = top_k['pLook'] * top_k['rating']
    pFound = top_k.groupby('qid')['pFound'].sum()
    
    return pFound.mean()

## Анализ результатов

Проверим скорость выполнения, а также рассчитанные значения метрики pFound с помощью решения с циклом и без цикла

In [57]:
# решение Yandex Cup
%timeit qid_pFound = qid_url_rating_hostid.groupby('qid').apply(pfound)

14 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
qid_pFound.mean()

0.4603173929969002

In [59]:
# решение без цикла
%timeit qid_pFound_new = pFound(qid_url_rating_hostid)

8.97 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [60]:
qid_pFound_new = pFound(qid_url_rating_hostid)
qid_pFound_new

0.4603173929969002

Можно заметить, что результат обеих функций одинаковый, что свидетельствует о правильности выполнения. Можно заметить, что у способа без циклов (используя возможности pandas) время выполнения ниже. 